### Finalizing Finance Data Dataset with Prices

The aim of this script is to combine the finance data set with prices.
The finance data set was created in a previous script, it contains data on fundementals and earnings of companies.
The prices data set contains daily prices of companies, set at the end of the day.

After merging the two files I will save it and move on the next step of the project.

In [21]:
# importing librariesq
import pandas as pd
import numpy as np
import os


In [22]:
# setting out the path for the data: Prices and Finance Data

prices_path = '../Data/Datasets/financial_data_prices.csv'

earnings_path = '../Data/Datasets/financial_data_EarningsCal.csv'

finance_path = '../Data/Datasets/starting_financial_data_combined.csv'


In [23]:
# loading the dataset

prices_data = pd.read_csv(prices_path)

earnings_data = pd.read_csv(earnings_path)

finance_data = pd.read_csv(finance_path)


In [ ]:
# viewing the data

# viewing the prices data

prices_data


In [ ]:
# viewing the finance data

finance_data.head(100)


In [ ]:
# viewing the earnings calendar data

earnings_data.head(100)

In [ ]:
# creating additional date columns to represent the date x + N from the earnings release date.
# first coluumn will be N = 1
# second column will be N = 7 (1 week)
# third column will be N = 28 (1 month)
# fourth column will be N = 91 (3 months)

earnings_data['date'] = pd.to_datetime(earnings_data['date'])

earnings_data['date_1'] = earnings_data['date'] + pd.DateOffset(days=1)

earnings_data['date_7'] = earnings_data['date'] + pd.DateOffset(days=7)

earnings_data['date_28'] = earnings_data['date'] + pd.DateOffset(days=28)

earnings_data['date_91'] = earnings_data['date'] + pd.DateOffset(days=91)


In [ ]:
earnings_data


In [ ]:
# for each of the date columns i would like to check if any of the dates fall on weekends. If they do I would like to move them to the next working day. I will do this check by creating additional columns that will provide the day of the week for each date column.

earnings_data['day_of_week'] = earnings_data['date'].dt.day_name()

earnings_data['day_of_week_1'] = earnings_data['date_1'].dt.day_name()

earnings_data['day_of_week_7'] = earnings_data['date_7'].dt.day_name()

earnings_data['day_of_week_28'] = earnings_data['date_28'].dt.day_name()

earnings_data['day_of_week_91'] = earnings_data['date_91'].dt.day_name()

# aggregate the day of the week columsn giving the count of each day of the week for each date column. The aim is to use pivot tables to see how the dates are distributed across the days of the week.

earnings_data.groupby('day_of_week_7').size() # in this case the 91 day column is used as an example. You can change the column to any of the date columns.


In [ ]:
# before getting the prices input into the earnings dataset, i will change the dates for any weekend dates to the next working day. Meaning for a statuday date, the date will be moved to the following monday (+2) and for a sunday date, the date will be moved to the monday (+1).

# Adjust weekend dates to the next working day
def adjust_weekend_dates(date):
    if date.weekday() == 5:  # Saturday
        return date + pd.DateOffset(days=2)
    elif date.weekday() == 6:  # Sunday
        return date + pd.DateOffset(days=1)
    else:
        return date


In [ ]:
# Apply the adjustment to each date column
date_columns = ['date', 'date_1', 'date_7', 'date_28', 'date_91']
for col in date_columns:
    earnings_data[col] = earnings_data[col].apply(adjust_weekend_dates)

# Display the adjusted earnings_data
earnings_data


In [ ]:
# for each of the date columns i would like to check if any of the dates fall on weekends. If they do I would like to move them to the next working day. I will do this check by creating additional columns that will provide the day of the week for each date column.

earnings_data['day_of_week'] = earnings_data['date'].dt.day_name()

earnings_data['day_of_week_1'] = earnings_data['date_1'].dt.day_name()

earnings_data['day_of_week_7'] = earnings_data['date_7'].dt.day_name()

earnings_data['day_of_week_28'] = earnings_data['date_28'].dt.day_name()

earnings_data['day_of_week_91'] = earnings_data['date_91'].dt.day_name()

# aggregate the day of the week columsn giving the count of each day of the week for each date column. The aim is to use pivot tables to see how the dates are distributed across the days of the week.

earnings_data.groupby('day_of_week_91').size() # in this case the 91 day column is used as an example. You can change the column to any of the date columns.

In [ ]:
# Adding the close price to the earnings data set based on the date and symbol (across all date columns)

# making sure that the date columns in the price dataset is a datetime object

prices_data['date'] = pd.to_datetime(prices_data['date'])


In [ ]:
# # testing looking up the close price for a single date and symbol in the prices data set
#
# filt_test_data = prices_data[(prices_data['symbol'] == 'ABNB') & (prices_data['date'] == '2020-09-30')]
#
# filt_test_data['close'].values[0]


In [ ]:
# merging the earnings data with the prices data

# function adds the close price to the earnings data based on the date and symbol

#def get_close_price(row, date_column):
  #  filtered_data = prices_data[(prices_data['symbol'] == row['symbol']) & (prices_data['date'] == row[date_column])]
    #if not filtered_data.empty:
      #  return filtered_data['close'].values[0]
    #else:
      #  return np.nan  # or any other default value you prefer

# Example usage:
# earnings_data['PriceEarningsData'] = earnings_data.apply(lambda row: get_close_price(row, 'date_1'), axis=1)


In [ ]:
# defining the function that adds the close price to the earnings data set based on the date and symbol

# def add_price_columns_and_save(earnings_data, prices_data, date_columns, save_filepath):
#     def get_close_price(row, date_column):
#         filtered_data = prices_data[(prices_data['symbol'] == row['symbol']) & (prices_data['date'] == row[date_column])]
#         if not filtered_data.empty:
#             return filtered_data['close'].values[0]
#         else:
#             return np.nan  # or any other default value you prefer
#
#     for date_column in date_columns:
#         price_column = f'price_{date_column}'
#         earnings_data[price_column] = earnings_data.apply(lambda row: get_close_price(row, date_column), axis=1)
#         # Save the DataFrame to a CSV file
#         earnings_data.to_csv(save_filepath, index=False)
#
#     return earnings_data

# trying a modified version of functio to lesson the number of NaN values

def add_price_columns_and_save(earnings_data, prices_data, date_columns, save_filepath):
    def get_close_price(row, date_column):
        for offset in range(4):  # Check up to day +3
            check_date = row[date_column] + pd.DateOffset(days=offset)
            filtered_data = prices_data[(prices_data['symbol'] == row['symbol']) & (prices_data['date'] == check_date)]
            if not filtered_data.empty:
                return filtered_data['close'].values[0]
        return np.nan  # or any other default value you prefer

    for date_column in date_columns:
        price_column = f'price_{date_column}'
        earnings_data[price_column] = earnings_data.apply(lambda row: get_close_price(row, date_column), axis=1)
        # Save the DataFrame to a CSV file
        earnings_data.to_csv(save_filepath, index=False)

    return earnings_data


In [ ]:
# testing out the functions on a smaller earnings data set based on 5 symbols and smaller date range

# making a copy of the earnings data set as a test set

earnings_data_test = earnings_data.copy()

# filtering the data set to only include the first 5 symbols

symbols = earnings_data_test['symbol'].unique()[:5]

earnings_data_test = earnings_data_test[earnings_data_test['symbol'].isin(symbols)]

# filtering the data set to only include dates between 2021-01-01 and 2021-01-31

earnings_data_test = earnings_data_test[(earnings_data_test['date'] >= '2023-01-01') & (earnings_data_test['date'] <= '2023-12-31')]

earnings_data_test


In [ ]:
# defining the inputs for the test function
save_filepath_test = '../Data/Datasets/updated_earnings_data_test.csv'

earnings_data_test = add_price_columns_and_save(earnings_data_test, prices_data, date_columns, save_filepath_test)

earnings_data_test

In [ ]:
# function has worked correctly, now to apply it to the full earnings data set
date_columns = ['date', 'date_1', 'date_7', 'date_28', 'date_91']
save_filepath = '../Data/Datasets/updated_earnings_data.csv'

earnings_data = add_price_columns_and_save(earnings_data, prices_data, date_columns, save_filepath)


In [24]:
# calling read_csv to check the data has been saved correctly as 'updated_earnings_data.csv'

updated_earnings_data = pd.read_csv('../Data/Datasets/updated_earnings_data.csv')

updated_earnings_data.head(10)


,date,symbol,eps,epsEstimated,time,revenue,revenueEstimated,updatedFromDate,fiscalDateEnding,date_1,...,day_of_week,day_of_week_1,day_of_week_7,day_of_week_28,day_of_week_91,price_date,price_date_1,price_date_7,price_date_28,price_date_91
0,2025-10-29,AAPL,NaN,NaN,amc,NaN,NaN,2024-12-09,2025-09-28,2025-10-30,...,Wednesday,Thursday,Wednesday,Wednesday,Wednesday,NaN,NaN,NaN,NaN,NaN
1,2025-07-30,AAPL,NaN,NaN,amc,NaN,NaN,2024-12-09,2025-06-28,2025-07-31,...,Wednesday,Thursday,Wednesday,Wednesday,Wednesday,NaN,NaN,NaN,NaN,NaN
2,2025-04-30,AAPL,NaN,NaN,amc,NaN,NaN,2024-12-09,2025-03-28,2025-05-01,...,Wednesday,Thursday,Wednesday,Wednesday,Wednesday,NaN,NaN,NaN,NaN,NaN
3,2025-02-06,AAPL,NaN,2.36,amc,NaN,NaN,2024-12-09,2024-12-28,2025-02-07,...,Thursday,Friday,Thursday,Thursday,Thursday,NaN,NaN,NaN,NaN,NaN
4,2024-10-31,AAPL,1.64,1.60,amc,9.493000e+10,9.451195e+10,2024-12-09,2024-09-28,2024-11-01,...,Thursday,Friday,Thursday,Thursday,Thursday,225.91,222.91,227.48,237.33,NaN
5,2024-08-01,AAPL,1.40,1.35,amc,8.577700e+10,8.443254e+10,2024-11-25,2024-06-29,2024-08-02,...,Thursday,Friday,Thursday,Thursday,Thursday,218.36,219.86,213.31,229.79,225.91
6,2024-05-02,AAPL,1.53,1.50,amc,9.075300e+10,9.036611e+10,2024-11-25,2024-03-30,2024-05-03,...,Thursday,Friday,Thursday,Thursday,Thursday,173.03,183.38,184.57,191.29,218.36
7,2024-02-01,AAPL,2.18,2.10,amc,1.195750e+11,1.179866e+11,2024-11-25,2023-12-30,2024-02-02,...,Thursday,Friday,Thursday,Thursday,Thursday,186.86,185.85,188.32,180.75,173.03
8,2023-11-02,AAPL,1.46,1.39,amc,8.949800e+10,8.417580e+10,2024-11-25,2023-09-30,2023-11-03,...,Thursday,Friday,Thursday,Thursday,Thursday,177.57,176.65,182.41,189.95,186.86
9,2023-08-03,AAPL,1.26,1.19,amc,8.179700e+10,8.168570e+10,2024-11-25,2023-07-01,2023-08-04,...,Thursday,Friday,Thursday,Thursday,Thursday,191.17,181.99,177.97,187.87,177.57


In [25]:
# removing rows where the 'date' is greater than 12M 2024 in order to not include these NAN values

updated_earnings_data = updated_earnings_data[updated_earnings_data['date'] <= '2024-12-01']



In [ ]:
# checking where the nan values are in the updated_earnings_data by company

# creating a pivot table where the rows are the symbols and the columns the count of nan values in each of the price_date columns

# Creating a pivot table to count NaN values in each price_date column by symbol
nan_pivot = updated_earnings_data[['symbol', 'price_date', 'price_date_1', 'price_date_7', 'price_date_28', 'price_date_91']].isna().groupby(updated_earnings_data['symbol']).sum()

# Display the pivot table
nan_pivot

In [26]:
# viewing data for specific symbols

#symbol to view

symbol = 'BABA'

# filtering the data to only include the symbol and the columns with nan values for the price_date columns

symbol_data = updated_earnings_data[updated_earnings_data['symbol'] == symbol][['date','symbol', 'day_of_week', 'day_of_week_1', 'day_of_week_7', 'day_of_week_28', 'day_of_week_91',
                                                                                'price_date', 'price_date_1', 'price_date_7', 'price_date_28', 'price_date_91']]

# selecting only rows that contain nan values

symbol_data = symbol_data[symbol_data.isna().any(axis=1)]

# displaying the data

symbol_data

,date,symbol,day_of_week,day_of_week_1,day_of_week_7,day_of_week_28,day_of_week_91,price_date,price_date_1,price_date_7,price_date_28,price_date_91
1510,2024-11-15,BABA,Friday,Monday,Friday,Friday,Friday,88.59,89.35,83.13,NaN,NaN
1552,2014-03-31,BABA,Monday,Tuesday,Monday,Monday,Monday,NaN,NaN,NaN,NaN,NaN
1553,2013-12-31,BABA,Tuesday,Wednesday,Tuesday,Tuesday,Tuesday,NaN,NaN,NaN,NaN,NaN
1554,2013-09-30,BABA,Monday,Tuesday,Monday,Monday,Monday,NaN,NaN,NaN,NaN,NaN
1555,2013-07-01,BABA,Monday,Monday,Monday,Monday,Monday,NaN,NaN,NaN,NaN,NaN


In [27]:
# merging the finance data with the updated earnings data. In the updated earnings data, I will only keep selected columns. Selections is made first.

# selecting the columns to keep in the updated earnings data

columns_to_keep = ['date', 'symbol', 'eps', 'epsEstimated',
                         'revenueEstimated', 'updatedFromDate', 'fiscalDateEnding', 'date_1',
                         'date_7', 'date_28', 'date_91', 'day_of_week', 'day_of_week_1',
                         'day_of_week_7', 'day_of_week_28', 'day_of_week_91', 'price_date',
                         'price_date_1', 'price_date_7', 'price_date_28', 'price_date_91']

updated_earnings_data = updated_earnings_data[columns_to_keep]

# viewing the data

updated_earnings_data.head(10)


,date,symbol,eps,epsEstimated,revenueEstimated,updatedFromDate,fiscalDateEnding,date_1,date_7,date_28,...,day_of_week,day_of_week_1,day_of_week_7,day_of_week_28,day_of_week_91,price_date,price_date_1,price_date_7,price_date_28,price_date_91
4,2024-10-31,AAPL,1.64,1.60,9.451195e+10,2024-12-09,2024-09-28,2024-11-01,2024-11-07,2024-11-28,...,Thursday,Friday,Thursday,Thursday,Thursday,225.91,222.91,227.48,237.33,NaN
5,2024-08-01,AAPL,1.40,1.35,8.443254e+10,2024-11-25,2024-06-29,2024-08-02,2024-08-08,2024-08-29,...,Thursday,Friday,Thursday,Thursday,Thursday,218.36,219.86,213.31,229.79,225.91
6,2024-05-02,AAPL,1.53,1.50,9.036611e+10,2024-11-25,2024-03-30,2024-05-03,2024-05-09,2024-05-30,...,Thursday,Friday,Thursday,Thursday,Thursday,173.03,183.38,184.57,191.29,218.36
7,2024-02-01,AAPL,2.18,2.10,1.179866e+11,2024-11-25,2023-12-30,2024-02-02,2024-02-08,2024-02-29,...,Thursday,Friday,Thursday,Thursday,Thursday,186.86,185.85,188.32,180.75,173.03
8,2023-11-02,AAPL,1.46,1.39,8.417580e+10,2024-11-25,2023-09-30,2023-11-03,2023-11-09,2023-11-30,...,Thursday,Friday,Thursday,Thursday,Thursday,177.57,176.65,182.41,189.95,186.86
9,2023-08-03,AAPL,1.26,1.19,8.168570e+10,2024-11-25,2023-07-01,2023-08-04,2023-08-10,2023-08-31,...,Thursday,Friday,Thursday,Thursday,Thursday,191.17,181.99,177.97,187.87,177.57
10,2023-05-04,AAPL,1.52,1.43,9.296020e+10,2024-11-25,2023-04-01,2023-05-05,2023-05-11,2023-06-01,...,Thursday,Friday,Thursday,Thursday,Thursday,165.79,173.57,173.75,180.09,191.17
11,2023-02-02,AAPL,1.88,1.94,1.213333e+11,2024-11-25,2022-12-31,2023-02-03,2023-02-09,2023-03-02,...,Thursday,Friday,Thursday,Thursday,Thursday,150.82,154.50,150.87,145.91,165.79
12,2022-10-27,AAPL,1.29,1.27,8.873928e+10,2024-11-25,2022-09-24,2022-10-28,2022-11-03,2022-11-24,...,Thursday,Friday,Thursday,Thursday,Thursday,144.80,155.74,138.88,148.11,143.96
13,2022-07-28,AAPL,1.20,1.16,8.279124e+10,2024-11-25,2022-06-25,2022-07-29,2022-08-04,2022-08-25,...,Thursday,Friday,Thursday,Thursday,Thursday,157.35,162.51,165.81,170.03,144.80


In [28]:
# merging the finance data with the updated earnings data, keeping all data from the finance data set and only the data from the updated earnings data set that matches the finance data set. The match will be based on date (date_x in finance dataset) and symbol

# making sure 'date_x'  an 'date' are both datetime objects

finance_data['date_x'] = pd.to_datetime(finance_data['date_x'])

updated_earnings_data['fiscalDateEnding'] = pd.to_datetime(updated_earnings_data['fiscalDateEnding'])


In [29]:
# viewing the data for the symbol 'AAPL' in the finance data set. Sorted by date_x

finance_data[finance_data['symbol'] == 'AAPL'].sort_values('date_x', ascending=False)


,addTotalDebt,assetGrowth,assetTurnover,averageInventory,averagePayables,averageReceivables,beta,bookValuePerShare,bookValueperShareGrowth,calendarYear,...,threeYDividendperShareGrowthPerShare,threeYNetIncomeGrowthPerShare,threeYOperatingCFGrowthPerShare,threeYRevenueGrowthPerShare,threeYShareholdersEquityGrowthPerShare,totalDebtToCapitalization,weightedAverageSharesDilutedGrowth,weightedAverageSharesGrowth,workingCapital,zip
0,106629000000,0.100624,0.260096,6.725500e+09,5.826700e+10,5.470750e+10,1.24,3.753628,-0.137951,2024,...,0.135642,-0.220800,0.442328,0.237508,-0.019076,0.651850,-0.006862,-0.009661,-2.340500e+10,95014
1,101304000000,-0.017187,0.258667,6.198500e+09,4.666350e+10,4.216100e+10,1.24,4.354308,-0.095859,2024,...,0.122352,0.070692,0.484993,0.143358,0.126469,0.602957,-0.007535,-0.005573,-6.189000e+09,95014
2,104590000000,-0.045551,0.268969,6.371500e+09,5.194950e+10,4.562600e+10,1.24,4.815961,0.008022,2024,...,0.170447,0.087751,0.028931,0.101665,0.166326,0.585008,-0.007186,-0.006699,4.594000e+09,95014
3,108040000000,0.002641,0.338247,6.421000e+09,6.037850e+10,5.554350e+10,1.24,4.777636,0.199247,2024,...,0.155970,0.287877,0.123787,0.171619,0.221760,0.593170,-0.006110,-0.005748,9.719000e+09,95014
4,123930000000,0.052367,0.253835,6.841000e+09,5.465500e+10,5.008550e+10,1.24,3.983862,0.037547,2023,...,0.170403,0.980735,0.147789,0.512628,0.040041,0.641260,-0.006505,-0.006254,-1.742000e+09,95014
5,109280000000,0.008664,0.244142,7.416500e+09,4.482200e+10,3.754250e+10,1.24,3.839692,-0.024779,2023,...,0.156923,0.941479,0.781654,0.506035,-0.083647,0.644514,-0.004545,-0.005672,-2.304000e+09,95014
6,109615000000,-0.042068,0.285513,7.151000e+09,5.043150e+10,4.503950e+10,1.24,3.937252,0.103066,2023,...,0.194736,1.372661,1.370283,0.796638,-0.124421,0.638139,-0.006811,-0.006643,-7.162000e+09,95014
7,111110000000,-0.017032,0.337866,5.883000e+09,6.101650e+10,5.755600e+10,1.24,3.569369,0.129191,2023,...,0.183114,0.499105,0.238259,0.417819,-0.295935,0.662011,-0.010097,-0.008587,-8.509000e+09,95014
8,120069000000,0.048901,0.255548,5.189500e+09,5.622900e+10,5.158700e+10,1.24,3.160998,-0.120742,2022,...,0.192725,0.696583,0.357916,0.577379,-0.372494,0.703223,-0.008839,-0.008202,-1.857700e+10,95014
9,119691000000,-0.040931,0.246675,5.446500e+09,5.051250e+10,4.382100e+10,1.24,3.595075,-0.131686,2022,...,0.187867,1.189526,1.225337,0.743911,-0.318580,0.673185,-0.008603,-0.007117,-1.758100e+10,95014


In [30]:
# viewing the data for the symbol 'AAPL' in the earnings data set. Sorted by fiscalDateEnding

updated_earnings_data[updated_earnings_data['symbol'] == 'AAPL'].sort_values('fiscalDateEnding', ascending=False)



,date,symbol,eps,epsEstimated,revenueEstimated,updatedFromDate,fiscalDateEnding,date_1,date_7,date_28,...,day_of_week,day_of_week_1,day_of_week_7,day_of_week_28,day_of_week_91,price_date,price_date_1,price_date_7,price_date_28,price_date_91
4,2024-10-31,AAPL,1.64,1.60,9.451195e+10,2024-12-09,2024-09-28,2024-11-01,2024-11-07,2024-11-28,...,Thursday,Friday,Thursday,Thursday,Thursday,225.9100,222.9100,227.4800,237.3300,NaN
5,2024-08-01,AAPL,1.40,1.35,8.443254e+10,2024-11-25,2024-06-29,2024-08-02,2024-08-08,2024-08-29,...,Thursday,Friday,Thursday,Thursday,Thursday,218.3600,219.8600,213.3100,229.7900,225.9100
6,2024-05-02,AAPL,1.53,1.50,9.036611e+10,2024-11-25,2024-03-30,2024-05-03,2024-05-09,2024-05-30,...,Thursday,Friday,Thursday,Thursday,Thursday,173.0300,183.3800,184.5700,191.2900,218.3600
7,2024-02-01,AAPL,2.18,2.10,1.179866e+11,2024-11-25,2023-12-30,2024-02-02,2024-02-08,2024-02-29,...,Thursday,Friday,Thursday,Thursday,Thursday,186.8600,185.8500,188.3200,180.7500,173.0300
8,2023-11-02,AAPL,1.46,1.39,8.417580e+10,2024-11-25,2023-09-30,2023-11-03,2023-11-09,2023-11-30,...,Thursday,Friday,Thursday,Thursday,Thursday,177.5700,176.6500,182.4100,189.9500,186.8600
9,2023-08-03,AAPL,1.26,1.19,8.168570e+10,2024-11-25,2023-07-01,2023-08-04,2023-08-10,2023-08-31,...,Thursday,Friday,Thursday,Thursday,Thursday,191.1700,181.9900,177.9700,187.8700,177.5700
10,2023-05-04,AAPL,1.52,1.43,9.296020e+10,2024-11-25,2023-04-01,2023-05-05,2023-05-11,2023-06-01,...,Thursday,Friday,Thursday,Thursday,Thursday,165.7900,173.5700,173.7500,180.0900,191.1700
11,2023-02-02,AAPL,1.88,1.94,1.213333e+11,2024-11-25,2022-12-31,2023-02-03,2023-02-09,2023-03-02,...,Thursday,Friday,Thursday,Thursday,Thursday,150.8200,154.5000,150.8700,145.9100,165.7900
12,2022-10-27,AAPL,1.29,1.27,8.873928e+10,2024-11-25,2022-09-24,2022-10-28,2022-11-03,2022-11-24,...,Thursday,Friday,Thursday,Thursday,Thursday,144.8000,155.7400,138.8800,148.1100,143.9600
13,2022-07-28,AAPL,1.20,1.16,8.279124e+10,2024-11-25,2022-06-25,2022-07-29,2022-08-04,2022-08-25,...,Thursday,Friday,Thursday,Thursday,Thursday,157.3500,162.5100,165.8100,170.0300,144.8000


In [31]:
# merging the finance data with the updated earnings data

merg_finance_data = pd.merge(finance_data, updated_earnings_data, how='left', left_on=['date_x', 'symbol'], right_on=['fiscalDateEnding', 'symbol'])


In [32]:
# viewing the merged data for the symbol 'AAPL' in the finance data set. Sorted by date_x

merg_finance_data[merg_finance_data['symbol'] == 'AAPL'].sort_values('date_x', ascending=False)

,addTotalDebt,assetGrowth,assetTurnover,averageInventory,averagePayables,averageReceivables,beta,bookValuePerShare,bookValueperShareGrowth,calendarYear,...,day_of_week,day_of_week_1,day_of_week_7,day_of_week_28,day_of_week_91,price_date,price_date_1,price_date_7,price_date_28,price_date_91
0,106629000000,0.100624,0.260096,6.725500e+09,5.826700e+10,5.470750e+10,1.24,3.753628,-0.137951,2024,...,Thursday,Friday,Thursday,Thursday,Thursday,225.91,222.91,227.48,237.33,NaN
1,101304000000,-0.017187,0.258667,6.198500e+09,4.666350e+10,4.216100e+10,1.24,4.354308,-0.095859,2024,...,Thursday,Friday,Thursday,Thursday,Thursday,218.36,219.86,213.31,229.79,225.91
2,104590000000,-0.045551,0.268969,6.371500e+09,5.194950e+10,4.562600e+10,1.24,4.815961,0.008022,2024,...,Thursday,Friday,Thursday,Thursday,Thursday,173.03,183.38,184.57,191.29,218.36
3,108040000000,0.002641,0.338247,6.421000e+09,6.037850e+10,5.554350e+10,1.24,4.777636,0.199247,2024,...,Thursday,Friday,Thursday,Thursday,Thursday,186.86,185.85,188.32,180.75,173.03
4,123930000000,0.052367,0.253835,6.841000e+09,5.465500e+10,5.008550e+10,1.24,3.983862,0.037547,2023,...,Thursday,Friday,Thursday,Thursday,Thursday,177.57,176.65,182.41,189.95,186.86
5,109280000000,0.008664,0.244142,7.416500e+09,4.482200e+10,3.754250e+10,1.24,3.839692,-0.024779,2023,...,Thursday,Friday,Thursday,Thursday,Thursday,191.17,181.99,177.97,187.87,177.57
6,109615000000,-0.042068,0.285513,7.151000e+09,5.043150e+10,4.503950e+10,1.24,3.937252,0.103066,2023,...,Thursday,Friday,Thursday,Thursday,Thursday,165.79,173.57,173.75,180.09,191.17
7,111110000000,-0.017032,0.337866,5.883000e+09,6.101650e+10,5.755600e+10,1.24,3.569369,0.129191,2023,...,Thursday,Friday,Thursday,Thursday,Thursday,150.82,154.50,150.87,145.91,165.79
8,120069000000,0.048901,0.255548,5.189500e+09,5.622900e+10,5.158700e+10,1.24,3.160998,-0.120742,2022,...,Thursday,Friday,Thursday,Thursday,Thursday,144.80,155.74,138.88,148.11,143.96
9,119691000000,-0.040931,0.246675,5.446500e+09,5.051250e+10,4.382100e+10,1.24,3.595075,-0.131686,2022,...,Thursday,Friday,Thursday,Thursday,Thursday,157.35,162.51,165.81,170.03,144.80


In [33]:

# saving the finance data as "updated_finance_data.csv"

merg_finance_data.to_csv('../Data/Datasets/updated_finance_data.csv', index=False)
